# Subproject D:

* Up until now, you have been computing what is known as term frequency: word counts are measures of frequency of the words (terms). What is more informative is term frequency inverse document frequency, or TF-IDF. This weights the term frequency by the inverse number of documents the word appears in.  
* The IDF term looks like this: $log(N / n_t) $, where $N$ is the number of documents (in this case, 8 books), and $n_t$ is the number of documents the specific word $t$ appears in (therefore, this should be between 1 and 8, inclusive).  
* You will also have to compute document-specific term frequencies (i.e., word $w_i$ appears $f_{i,j}$ times in document $j$, but $f_{i,k}$ times in document $k$; you’ll need both those counts!). Then, you multiply the IDF term by each document-specific TF term, and that gives you the TF-IDF score for each document.  
* In this case, your output should still contain 40 words, but should contain the top-5 words in each document by their TF-IDF rank 

## Algorithm
* We define the TF-IDF for word $w_i$ in document $j$ as  
\begin{equation}
TF\text{-}IDF_{i, j} = f_{i, j} \cdot log(\frac {N} {n_j})
\end{equation}
where $n_j = 0, 1, 2, ... N$. In the case, $N = 8$.
* It is intuitive to create a 2-dimensional array, $A$, to store the frequency for each word in each specific document, where  
\begin{equation}
A[i][j] = \text{ frequency (times of occurrence) for word, } w_i \text{, in document } j
\end{equation}

In [1]:
from pyspark import SparkContext, SparkConf

In [2]:
import json
import os
import numpy as np

In [3]:
conf = SparkConf().setAppName('word count').setMaster('local[3]')
sc = SparkContext(conf = conf)

In [4]:
stoplist = sc.textFile('./data/stopwords.txt')
broadcastStopList = sc.broadcast(set(stoplist.collect()))

In [5]:
punctuations = sc.broadcast(set(".,:;'!?"))

* Read all text through wholeTextFiles() method, which will return a key-value pair, where the key is the path of each file, the value is the content of each file  
* Create a map, where the key is the path of the file, and the value is the index, i.e. 0, 1, ... 7
* Split each line into a list of words  
* Filter out the words which are in the stopwords list  
* Map each word, w, to a tuple, (w, [1, 0, 0, ... 0]). For the second document, the tuple would be (w, [0, 1, 0, ... 0]). etc    
* Add the tuples with the same key (word)  
* Stripping the words which have leading or trailing punctuations  

In [6]:
textFiles = sc.wholeTextFiles('./data/4300-0.txt,./data/pg*.txt')

In [95]:
numFiles = textFiles.count()
filenames = textFiles.map(lambda textfile : textfile[0]).collect()
fileToIdx = {}
for i, filename in enumerate(filenames):
    increment = [0 for _ in range(numFiles)]
    increment[i] = 1
    fileToIdx[filename] = increment
broadcastFileToIdx = sc.broadcast(fileToIdx)

In [104]:
lines = textFiles.flatMap(lambda textfile : [(textfile[0], line) for line in textfile[1].splitlines()])

In [165]:
words = lines.flatMap(lambda pathLine : [(word.casefold(), broadcastFileToIdx.value[pathLine[0]]) for word in pathLine[1].split()])

In [202]:
filteredPunctuation = words.filter(lambda wordIncre : len(wordIncre[0]) > 1).map(lambda wordIncre : (wordIncre[0][1:], wordIncre[1]) if len(wordIncre[0]) > 0 and wordIncre[0][0] in punctuations.value else wordIncre).map(lambda wordIncre : (wordIncre[0][:-1], wordIncre[1]) if len(wordIncre[0]) > 0 and wordIncre[0][-1] in punctuations.value else wordIncre)

In [185]:
filteredPunctuation = words.map(lambda wordIncre : (wordIncre[0].strip(".,:;'!?"), wordIncre[1]))

In [203]:
filteredWords = filteredPunctuation.filter(lambda wordIncre : wordIncre[0] not in broadcastStopList.value)

In [204]:
counts = filteredWords.reduceByKey(lambda a, b : [a[k] + b[k] for k in range(numFiles)])

Calculate the TF-IDF based on the definition above-mentioned

In [205]:
tfidf = counts.map(lambda x : (x[0], [k * np.log(numFiles / sum(j > 0 for j in x[1])) for k in x[1]]))

In [206]:
tfidf.take(5)

[('project', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
 ('gutenberg', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
 ('ebook', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]),
 ('james',
  [32.367365349386965,
   0.0,
   0.0,
   0.0,
   3.9233170120469048,
   3.9233170120469048,
   0.0,
   0.0]),
 ('use', [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])]

In [207]:
output = {}
for i in range(numFiles):
    top5 = tfidf.filter(lambda x : x[1][i] > 2).takeOrdered(5, key=lambda x : -x[1][i])
    for j in range(5):
        output[top5[j][0]] = top5[j][1][i]

In [208]:
output

{'stephen': 1000.211381548001,
 'bloom': 614.12840197611149,
 'j': 365.98171133565108,
 'don’t': 361.8228282522914,
 'dedalus': 332.71064666877373,
 'republic': 228.73856958478194,
 'glaucon': 178.83197258446589,
 'thrasymachus': 178.83197258446589,
 'plato': 151.81117224637262,
 'adeimantus': 149.71979100094816,
 'alice': 230.12486394590184,
 '[illustration]': 45.747713916956386,
 'rabbit': 29.424877590351784,
 'mouse': 23.539902072281429,
 'caterpillar': 19.408121055678468,
 'martians': 330.63120512709389,
 'martian': 106.74466580623158,
 'woking': 103.97207708399179,
 'pit': 77.485510987926375,
 'heat-ray': 70.701012417114413,
 'jo': 1651.0765840937897,
 'meg': 1276.7771065914192,
 'amy': 1139.53396484055,
 'laurie': 1089.627367840234,
 'beth': 540.65480083675732,
 'soveraign': 1023.0852385064792,
 'common-wealth': 867.12712288049147,
 'onely': 856.72991517209232,
 'civill': 675.81850104594662,
 'kingdome': 592.64083937875318,
 'hector': 500.4522643642805,
 'thy': 437.10337519853414

In [209]:
with open('sp4.json', 'w') as outfile:
    json.dump(output, outfile)

In [ ]:
sc.stop()